# AUTIN EXPORT FILES TREATMENT

In [ ]:
# import pandas as pd
import os
import pandas as pd
import glob

# Récupération des fichiers

paths = []

for root, dirs, files in os.walk(r"D:\FO_RAN\REPORTING\EXTRACTION\AUTIN_EXTRACTION"):
    for file in files:
        if file.endswith(".xlsx"):
             s = os.path.join(root, file)
             print(s)
             paths.append(s)
       
    
all_data = pd.DataFrame()
for f in paths:
    df = pd.read_excel(f)
    all_data = all_data.append(df, ignore_index = False)
    
    
writer = pd.ExcelWriter('D:\\FO_RAN\\REPORTING\\EXTRACTION\\AUTIN_EXTRACTION\\HISTORIQUE_AUTIN.xlsx')
all_data.to_excel(writer,'Alarms', index = False)
writer.save()

# NETACT CSV EXPORT FILES TREATMENT

In [ ]:
import pandas as pd
import os
import glob
import csv

# Récupération des fichiers

paths = []

for root, dirs, files in os.walk(r"D:\REPORTING\EXTRACTION"):
    for file in files:
        if file.endswith(".csv"):
             s = os.path.join(root, file)
             print(s)
             paths.append(s)
       
    
all_data = pd.DataFrame()
for f in paths:
    df = pd.read_csv(f, sep = ';')
    all_data = all_data.append(df, ignore_index = False)
    
    
writer = pd.ExcelWriter('D:\\REPORTING\\EXTRACTION\\HISTORIQUE.xlsx')
all_data.to_excel(writer,'Alarms', index = False)
writer.save()

# RECHERCHE CAUSE AUTIN

In [14]:
import pandas as pd

import numpy as np

History = pd.read_excel(r'D:\FO_RAN\REPORTING\EXTRACTION\HISTORY\HISTORIQUE_WK_2G3G4G.xlsx')

AUTIN = pd.read_excel('D:\\FO_RAN\\REPORTING\\EXTRACTION\\HISTORY\\HISTORIQUE_AUTIN.xlsx', usecols =["EMS Serial Number", "First Occurrence", "BizReserve Field27", "BizReserve Field28", "Extended Info"])

AUTIN.rename(columns={"BizReserve Field27":"RootCause", "BizReserve Field28":"AUTIN_Cause"}, inplace=True)
    
inner_join = History.set_index('Consecutive Number').join(AUTIN.set_index('EMS Serial Number'))

inner_join.sort_values(by='Alarm Time', ascending=True, inplace=True)

writer =  pd.ExcelWriter('D:\\FO_RAN\\REPORTING\\EXTRACTION\\HISTORY\\HISTORIQUE_WK_AUTINCAUSE_2G3G4G.xlsx')

History.to_excel(writer,'History_Alarms', index = False)
                     
writer.save()

# RECHERCHE CAUSE WFM

In [1]:
import pandas as pd

import numpy as np

History = pd.read_excel(r'D:\FO_RAN\REPORTING\OTHERS\OTHER\HISTORIQUE_WK_2G3G4G.xlsx')

WFM = pd.read_excel('D:\\FO_RAN\\REPORTING\\EXTRACTION\\HISTORY\\HISTORIQUE_WFM.xlsx', usecols =["Site ID", "Fault First Occur Time", "SubCause", "RCA L2"])

CODE_1 = History["Name"].str.split('_', expand= True)[0]

CODE_2 = WFM["Site ID"].str.split('_', expand= True)[0]

DATE_1 = [d.date() for d in History['Alarm Time']]

DATE_2 = [d.date() for d in WFM['Fault First Occur Time']]

WFM.rename(columns={"SubCause":"WFM_Cause"}, inplace=True)

History.insert(1, "CODE", CODE_1)

WFM.insert(0, "CODE", CODE_2)

History.insert(2, "DATE", DATE_1)

WFM.insert(1, "DATE", DATE_2)
    
inner_join = pd.merge(History, WFM, on=['CODE', 'DATE'], how='left')

inner_join.drop_duplicates(subset=['Consecutive Number','Alarm Time' ,'Cancel Time', 'DESCRIPTIONS', 'CAUSES'], inplace=True, keep='first')

inner_join.sort_values(by='Alarm Time', ascending=True, inplace=True)
    
#History["DESCRIPTIONS"].update(inner_join["WFM_Cause"])

writer =  pd.ExcelWriter('D:\\FO_RAN\\REPORTING\\EXTRACTION\\HISTORY\\HISTORIQUE_WK_WFMCAUSE_2G3G4G.xlsx')

inner_join.to_excel(writer,'History_Alarms', index = True)
                     
writer.save()

# RECHERCHE CAUSES DUES AUX TRAVAUX

In [6]:
import pandas as pd

import os

import numpy as np

History = pd.read_excel(r'D:\FO_RAN\REPORTING\OTHERS\OTHER\HISTORIQUE_WK_2G3G4G.xlsx')

Travaux = pd.read_excel(r'D:\FO_RAN\REPORTING\EXTRACTION\TRAVAUX PROGRAMMES ET TRAVAUX NOKIA.xlsx')

BTS = History["Name"].str.replace("_S_", "_")

SITE = BTS.str.split('_', expand= True)[1]

Causes = Travaux["TRAVAUX"].str.split(':', expand=True)[1]

Sites = Travaux["TRAVAUX"].str.split(':', expand=True)[0]

DATE = [d.date() for d in Travaux['DATE_TIME']]

DATE_1 = [d.date() for d in History['Alarm Time']]

History.insert(2, "SITE", SITE)

History.insert(2, "DATE", DATE_1)

Travaux.insert(2, "WORK_Cause", Causes)

Travaux.insert(0, "SITE", Sites)

Travaux.insert(1, "DATE", DATE)

Travaux.drop(columns="TRAVAUX", axis=1, inplace= True)

Travaux.drop(columns="DATE_TIME", axis=1, inplace= True)

inner_join = pd.merge(History, Travaux, on=['SITE', 'DATE'], how='left')

inner_join.drop_duplicates(subset=['Consecutive Number','Alarm Time', 'Cancel Time', 'DESCRIPTIONS', 'CAUSES'], inplace=True, keep='first')

inner_join.sort_values(by='Alarm Time', ascending=True, inplace=True)

History["DESCRIPTIONS"].update(inner_join["WORK_Cause"])

writer =  pd.ExcelWriter('D:\\FO_RAN\\REPORTING\\\\EXTRACTION\\HISTORY\\HISTORIQUE_WORK.xlsx')

History.to_excel(writer,'History_Alarms', index = True)

writer.save()

AttributeError: partially initialized module 'pandas' has no attribute 'core' (most likely due to a circular import)

# METEO ROOT CAUSE

In [3]:
import pandas as pd

import numpy as np

History = pd.read_excel(r'D:\FO_RAN\REPORTING\EXTRACTION\HISTORY\HISTORIQUE_WK_2G3G4G.xlsx')

METEO = pd.read_excel('D:\\FO_RAN\\REPORTING\\EXTRACTION\\HISTORY\\METEO_WK.xlsx', usecols =["Sites hors-services", "Cause", "Entité Responsable", "Heures de coupures", "Ticket WFM"])

BTS = History["Name"].str.replace("_S_", "_")

SITE = BTS.str.split('_', expand= True)[1]

History.insert(2, "SITE", SITE)

METEO.rename(columns={"Sites hors-services":"SITE", "Cause":"METEO_Cause"}, inplace=True)

DATE = [d.date() for d in METEO['Heures de coupures']]

DATE_1 = [d.date() for d in History['Alarm Time']]

METEO.insert(1, "DATE", DATE)

History.insert(2, "DATE", DATE_1)
    
inner_join = pd.merge(History, METEO, on=['SITE', 'DATE'], how='left')

inner_join.drop_duplicates(subset=['Consecutive Number','Alarm Time', 'Cancel Time', 'DESCRIPTIONS', 'CAUSES'], inplace=True, keep='first')

inner_join.sort_values(by='Alarm Time', ascending=True, inplace=True)

writer =  pd.ExcelWriter('D:\\FO_RAN\\REPORTING\\EXTRACTION\\HISTORY\\HISTORIQUE_WK_METEO.xlsx')

inner_join.to_excel(writer,'History_Alarms', index = True)
                     
writer.save()  

# CONSOLIDATION DES FICHIERS HISTORIQUES

In [7]:
import pandas as pd

import numpy as np

History_1 = pd.read_excel(r'D:\FO_RAN\REPORTING\OTHERS\OTHERS\RETOUR IHS.xlsx')

History_2 = pd.read_excel(r'D:\FO_RAN\REPORTING\OTHERS\OTHERS\RETOUR CAMUSAT.xlsx', usecols =["Consecutive Number", "DESCRIPTIONS", "Heure de Coupure CIE", "TYPOLOGIES POWER"])
 
History_1.update(History_2 'Consecutive Number', lsuffix='_l', rsuffix='_r')

writer =  pd.ExcelWriter('D:\\FO_RAN\\REPORTING\\OTHERS\\OTHERS\\RETOUR IHS & CAMUSAT.xlsx')

inner_join.to_excel(writer,'History_Alarms', index = True)
                     
writer.save()